In [ ]:
import numpy as np

import nbimporter
import m01_profiles as profiles
import m02_elections as elections
import m03_weighting as weighting
import m04_issue_voting as voting

# TO DO
- Compute agreements for all election rules given and save data to file
- Compute means and variances of the agreements for every rule
- For each independent variable, create a plot comparing the agreement of each rule under RD
- Do the same for FRD for each delegation type

In [ ]:
def agreements_rd(n_iter, n_voters, n_cands, n_issues, voters_p, cands_p, rules_list):
    agreements = {rule:[] for rule in rules_list}
    
    for i in range(n_iter):
        v_profile, c_profile, vc_distances = profiles.generate_instance(n_voters, 
                                                                        n_cands, 
                                                                        n_issues, 
                                                                        voters_p, 
                                                                        cands_p, 
                                                                        seed=i)
        #print(f'v_profile: {v_profile}')
        #print(f'c_profile: {c_profile}')
        #print(f'vc_distances: {vc_distances}')
        voter_majority_outcomes = voting.majority(v_profile)
#         print(f'voter majority outcomes: {voter_majority_outcomes}')

        derivatives = profiles.all_issues_profile_derivatives(v_profile, c_profile, threshold)

        for rule in rules_list:
            if rule == 'max_approval':
                election_profile = derivatives['threshold_approval_profile']
                rep_ids, _ = elections.max_approval(election_profile, n_reps, seed=i)
            elif rule == 'rav':
                election_profile = derivatives['threshold_approval_profile']
                rep_ids = elections.repeated_alternative_vote(election_profile, n_reps) #no seed, old implementation
            elif rule == 'borda':
                election_profile = derivatives['ordermaps_profile']
                rep_ids, _ = elections.borda(election_profile, n_reps, seed=i)
            elif rule == 'plurality':
                election_profile = derivatives['ordermaps_profile']
                rep_ids, _ = elections.plurality(election_profile, n_reps, seed=i)
#             elif rule == 'stv':
#                 election_profile = derivatives['orders_profile']
#                 rep_ids = elections.single_transferable_vote(election_profile, n_reps, seed=i)
            elif rule == 'max_score':
                election_profile = derivatives['scores_profile']
                rep_ids, _ = elections.max_score(election_profile, n_reps, seed=i)
            else:
                raise ValueError(f'Rule {rule} not implemented yet in this experiment')
                
            #print(f'Using rule: {rule}')

            given_weights = weighting.set_default_given_weights('uniform', election_profile, list(range(n_cands)), rep_ids, normalized = True)
            weights = weighting.aggregate_weights(given_weights)
            issue_outcomes = voting.weighted_majority(c_profile, weights)
            #print(f'weights: {weights}')
            #print(f'disagreements: {(voter_majority_outcomes ^ issue_outcomes)}')
            dd_rd_distance = np.sum(voter_majority_outcomes ^ issue_outcomes) / n_issues
            agreements[rule].append(1-dd_rd_distance)

    return agreements
            
            

# Quick Test

In [ ]:
n_iter = 10000
n_voters = 1
n_cands = 1
n_reps = 1
n_issues = 1000
threshold = 0.5

voters_p = 0.7
cands_p = 0.2

rules_list = ['max_approval','borda', 'plurality', 'rav','max_score']#, 'stv']

agreements = agreements_rd(n_iter, n_voters, n_cands, n_issues, voters_p, cands_p, rules_list)
print(agreements)

{'max_approval': [0.378, 0.382, 0.361, 0.349, 0.375, 0.385, 0.394, 0.387, 0.363, 0.363, 0.378, 0.38, 0.379, 0.368, 0.371, 0.378, 0.363, 0.373, 0.34099999999999997, 0.373, 0.373, 0.379, 0.369, 0.355, 0.38, 0.387, 0.368, 0.38, 0.405, 0.357, 0.391, 0.359, 0.398, 0.363, 0.374, 0.393, 0.352, 0.374, 0.376, 0.382, 0.388, 0.378, 0.388, 0.38, 0.39, 0.364, 0.401, 0.401, 0.394, 0.384, 0.364, 0.34299999999999997, 0.405, 0.376, 0.387, 0.381, 0.359, 0.387, 0.361, 0.369, 0.398, 0.379, 0.383, 0.37, 0.359, 0.363, 0.385, 0.387, 0.375, 0.388, 0.373, 0.389, 0.357, 0.361, 0.397, 0.384, 0.346, 0.403, 0.373, 0.363, 0.4, 0.37, 0.362, 0.401, 0.368, 0.35, 0.381, 0.381, 0.388, 0.393, 0.356, 0.379, 0.379, 0.389, 0.382, 0.34199999999999997, 0.384, 0.396, 0.349, 0.384, 0.379, 0.363, 0.389, 0.41100000000000003, 0.388, 0.361, 0.39, 0.396, 0.38, 0.393, 0.398, 0.371, 0.347, 0.374, 0.381, 0.406, 0.372, 0.369, 0.366, 0.40700000000000003, 0.39, 0.377, 0.383, 0.383, 0.353, 0.384, 0.368, 0.362, 0.385, 0.374, 0.382, 0.375, 0

In [ ]:
#Single Run Params and (Variables)

#random seed (just set once at very beginning)

#n_voters
#n_cands
#n_issues
#voters_p
#cands_p

#n_reps

#election_profile_derivation (approvals, orders/ordermaps/ranks, scores)
    ## better to use dict with order (not ordermap) so pref orders can be incomplete
#(election_profile)

#election_rule
#(rep_ids (or indicator array))

#default_style

#n_delegators
#(delegator_ids)
#delegation_style (top k, approval, incisive)(only incisive is issue-by-issue)

#(given_weights) (3D array if issue-by-issue)(n_voters x n_cands x n_issues)

#weight_aggregation (just use sum)
#weighted majority voting rule

#(issue_outcomes)
#(agreement_with_dd)

#n_iter

In [ ]:
#Conclusion: 

#1. Use 3 classes: profile, election, and delegative_voting
#2. Use dicts for election_profiles

#profile: issues profiles, distances, and induced derivatives
#election: cands, voters, n_reps, election_prefs, election_rule
#delegative_voting: default weights, delegation style, v_issue_prefs, c_issue_prefs, delegators, reps